In [1]:
!git clone https://github.com/coangquang/zalo2021.git
%cd zalo2021
!mkdir generated_data

Cloning into 'zalo2021'...
remote: Enumerating objects: 2272, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (259/259), done.
remote: Total 2272 (delta 113), reused 196 (delta 46), pack-reused 1963
Receiving objects: 100% (2272/2272), 10.54 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (642/642), done.
/kaggle/working/zalo2021


In [2]:
#!pip install --upgrade pip
#!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
#!pip install transformers==4.9.1

In [3]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key")
wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
#!python create_corpus.py --data_dir /kaggle/input/ltr2021

In [5]:
!mkdir generated_data/condenser

In [6]:
!python Condenser/helper/create_train.py --tokenizer_name vinai/phobert-base-v2 --file /kaggle/input/retraining/retrain.json --save_to generated_data/condenser --max_len 256 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|████████████████████████████████████| 60795/60795 [01:22<00:00, 738.49it/s]


In [7]:
!mkdir saved_model

In [8]:
!python Condenser/run_pre_training.py \
  --output_dir saved_model \
  --model_name_or_path /kaggle/input/phobert-v2-9 \
  --do_train \
  --save_steps 1000 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 4 \
  --fp16 \
  --warmup_ratio 0.1 \
  --learning_rate 5e-5 \
  --num_train_epochs 8 \
  --overwrite_output_dir \
  --dataloader_num_workers 2\
  --n_head_layers 2 \
  --skip_from 6 \
  --max_seq_length 256 \
  --train_dir generated_data/condenser \
  --weight_decay 0.01 \
  --late_mlm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Parse else mark
Parse done
Before train dataset ['generated_data/condenser/condenser_corpus_vectors.json']
Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1056.50it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6294bb6a9a353517/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 135.31it/s]
Before validate dataset None
[INFO|configuration_utils.py:713] 2023-10-04 10:58:36,741 >> loading configuration file /kaggle/input/phobert-v2-9/config.json
[INFO|configuration_utils.py:775] 2023-10-04 10:58:36,742 >> Model config RobertaConfig {
  "_name_or_path": "